<a href="https://colab.research.google.com/github/willylao59/Hackathon_NER/blob/master/hackaton_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
PATH = "/content/drive/My Drive/data_hackaton_NER/"

import pandas as pd
import numpy as np

aircraft    = pd.read_pickle(PATH + 'Aircraft.pkl')
text        = pd.read_pickle(PATH + 'text.pkl')

In [0]:
train_list  = 'APPROX 5 MI NW OF THE MISSION BAY VOR'.split(' ')
train_label = ['O', 'aircraft', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
test_list   = 'APPROX 5 MI NW OF THE MISSION BAY VOR'.split(' ')

train = pd.DataFrame({'word': train_list, 'train_label': train_label})
test  = pd.DataFrame({'word':test_list})

np.savetxt(PATH + "train.txt", train.values, fmt=)
np.savetxt(PATH + "text.txt", test.values)

TypeError: ignored

In [82]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0: 'text', 1: 'aircraft'}

# this is the folder in which train, test and dev files reside
data_folder = PATH

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train_file.txt',
                              test_file='test_file.txt')

2020-01-22 14:06:10,950 Reading data from /content/drive/My Drive/data_hackaton_NER
2020-01-22 14:06:10,952 Train: /content/drive/My Drive/data_hackaton_NER/train_file.txt
2020-01-22 14:06:10,953 Dev: None
2020-01-22 14:06:10,954 Test: /content/drive/My Drive/data_hackaton_NER/test_file.txt


In [84]:
len(corpus.train)
print(corpus.train[0].to_tagged_string('word'))
print(corpus.train[1].to_tagged_string('aircraft'))

A B737-300 DURING A RELEASE FROM CONTRACT MAINT INSPECTION, A TECHNICIAN DISCOVERED UNAPPROVED GREEN TAPE WRAPPED ON FLT CTL CABLES LOCATED ABOVE #2 AIR DATA COMPUTER
B737 <aircraft> FLC FORGOT TO PUT THE COCKPIT JUMP SEAT DEPLOYED PRIOR TO PUSH BACK DUE TO DISTRACTION OF EXPECTING TO CHECK RIDER CREDENTIALS PRIOR TO TKOF


In [0]:
from flair.datasets import WNUT_17
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings
from typing import List

# 1. get the corpus
print(corpus)

# 2. what tag do we want to predict?
tag_type = 'aircraft'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

# 4. initialize embeddings
embedding_types: List[TokenEmbeddings] = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

# 8. plot weight traces (optional)
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_weights('resources/taggers/example-ner/weights.txt')

Corpus: 19559 train + 2173 dev + 5434 test sentences
[b'<unk>', b'O', b'aircraft', b'<START>', b'<STOP>']


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2020-01-22 14:10:31,872 ----------------------------------------------------------------------------------------------------
2020-01-22 14:10:31,873 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=5, bias=True)
)"
2020-01-22 14:10:31,873 ----------------------------------------------------------------------------------------------------
2020-01-22 14:10:31,877 Corpus: "Corpus: 19559 train + 2173 dev + 5434 test sentences"
2020-01-22 14:10:31,878 ----------------------------------------------------------------------------------------------------
2020-01-22 14:10:31,880 Parameters:
2020-01-22 14:10:31,882  - learning_rate: "0.1"
2020-01-22 14:10:31,883  - mini_batch_siz

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


2020-01-22 14:11:43,465 ----------------------------------------------------------------------------------------------------
2020-01-22 14:11:43,555 epoch 2 - iter 0/612 - loss 0.31280625 - samples/sec: 22399.36
2020-01-22 14:11:48,873 epoch 2 - iter 61/612 - loss 0.55731987 - samples/sec: 368.04
2020-01-22 14:11:54,010 epoch 2 - iter 122/612 - loss 0.55258312 - samples/sec: 381.07
2020-01-22 14:11:59,208 epoch 2 - iter 183/612 - loss 0.55227934 - samples/sec: 376.62
2020-01-22 14:12:04,406 epoch 2 - iter 244/612 - loss 0.55033369 - samples/sec: 376.71
2020-01-22 14:12:09,434 epoch 2 - iter 305/612 - loss 0.53893211 - samples/sec: 389.46
2020-01-22 14:12:14,704 epoch 2 - iter 366/612 - loss 0.53317045 - samples/sec: 371.65
2020-01-22 14:12:19,686 epoch 2 - iter 427/612 - loss 0.52909539 - samples/sec: 393.00
2020-01-22 14:12:24,802 epoch 2 - iter 488/612 - loss 0.52362345 - samples/sec: 382.80
2020-01-22 14:12:30,002 epoch 2 - iter 549/612 - loss 0.52319922 - samples/sec: 376.56
2020-0

In [0]:
# load the model you trained
model = SequenceTagger.load('resources/taggers/example-ner/final-model.pt')

# create example sentence
sentence = Sentence('I love Berlin at A300 et Beluga XL laze layx')

# predict tags and print
model.predict(sentence)

print(sentence.to_tagged_string())